# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Program: _Computer Systems Engineering_  </center>
---
### <center> **Spring 2025** </center>
---

**Lab 05**: TikTok tracks analysis.

**Date**: 24/02/2025

**Team Name**: Par De Dos

**Professor**: Pablo Camarillo Ramirez

In [1]:
# Initialize findspark to get acces to de PySpark installation
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

# Create connection to the spark cluster
spark = SparkSession.builder \
    .appName("Lab 05 - Par de Dos") \
    .master("spark://368ad5a83fd7:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()

# Create SparkContext
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/25 01:26:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
import importlib
import team_ParDeDos.spark_utils

importlib.reload(team_ParDeDos.spark_utils)

columns_info = [ ("track_id", "string"),
                ("track_name", "string"),
                ("artist_id", "string"),
                ("artist_name", "string"),
                ("album_id", "string"),
                ("duration", "integer"),
                ("release_date", "timestamp"),
                ("popularity", "integer"),
                ("danceability", "double"),
                ("energy", "double"),
                ("key", "integer"),
                ("loudness", "double"),
                ("mode", "integer"),
                ("speechiness", "double"),
                ("acousticness", "double"),
                ("instrumentalness", "double"),
                ("liveness", "double"),
                ("valence", "double"),
                ("tempo", "double"),
                ("playlist_id", "string"),
                ("playlist_name", "string"),
                ("duration_mins", "double"),
                ("genre", "string")]

schema = team_ParDeDos.spark_utils.SparkUtils.generate_schema(columns_info)

# Create DataFrame
tiktok_df = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/tiktok.csv")

In [9]:
# Filter and count popular tracks. Filter songs with a popularity 
# score greater than 80 and count the number of tracks

filtered_tiktok_df = tiktok_df.filter(tiktok_df["popularity"] > 80)

row_count = filtered_tiktok_df.count()
row_count

1023

In [17]:
# Find the top 5 most energetic songs. Sort songs by energy in 
# descending order and retrieve the top 5 songs
sorted_tiktok_df = tiktok_df.orderBy(columns_info["energy"].desc())

sorted_tiktok_df.show(n=5)

TypeError: list indices must be integers or slices, not str

In [18]:
# Stop the SparkContext
sc.stop()